In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%aiida

from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.base import Int, Str, Float
from aiida.orm.data.remote import RemoteData
from aiida.work import workfunction
from aiida.work.process import WorkCalculation
from aiida.work.run import submit
from aiida_cp2k.calculations import Cp2kCalculation

import ase
import ase.io
import numpy as np
import nglview
from copy import deepcopy
from pprint import pprint

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

from apps.reactions.replicawork import ReplicaWorkchain

In [ ]:
from tempfile import NamedTemporaryFile
from base64 import b64encode
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

def display_thumbnail(th):
    return ipw.HTML('<img width="400px" src="data:image/png;base64,{}" title="">'.format(th))

# Generate Replicas

Two kinds of collective variable are implemented:

**DISTANCE between two atoms**
- Select the two atoms using the "Apply (O)" button to visualize the selected atoms as oxygen (red)

**ANGLE_PLANE_PLANE between two planes**
- Define two planes by atoms or a vector
- If the atoms lie in a plane parallel to the xy-plane: ~~Choose the three atoms in counter-clockwise order (viewed from the top) and the angle wrt. to the z-axis will be 0deg.~~ Just evaluate the collective variable. The preview for now only computes the angle wrt. (0, 0, 1).

## Step 1: Select initial structure

In [ ]:
from apps.surfaces.structure_browser import StructureBrowser

def on_struct_change(c):
    refresh_structure_view()
    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()

clear_output()
display(ipw.VBox([struct_browser, viewer]))
def refresh_structure_view():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    node = struct_browser.results.value
    if not node:
        return
    atoms = node.get_ase()
    # Fixed atoms as F
    try:
        bounds = map(int, text_fixed_atoms.value.split('..'))
        for a in range(bounds[0]-1, bounds[1]):
            atoms[a].symbol = 'F'
    except:
        pass
    
    # COLVAR atoms as O
    a_list = text_colvar_atoms.value.split()
    for a in a_list:
        atoms[int(a)-1].symbol = 'O'
    #if len(a_list) == 2:
    #    print 'Distance between the atoms:'
    #    d = t[int(a_list[0])-1].position-t[int(a_list[1])-1].position
    #    print d
    #    print np.linalg.norm(d)
    
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()

    spin_ups = ",".join([str(i) for i, k in enumerate(node.get_site_kindnames()) if k.endswith("1")])
    spin_downs = ",".join([str(i) for i, k in enumerate(node.get_site_kindnames()) if k.endswith("2")])

    viewer.add_representation('ball+stick', selection="@"+spin_ups, color='red', aspectRatio=3.0, opacity=0.4)
    viewer.add_representation('ball+stick', selection="@"+spin_downs, color='green', aspectRatio=3.0, opacity=0.4)

## Step 2: General parameters of the calculation

- The replica name will be visible in the search, so pick something descriptive
- Leave the input "Cell" empty in order to use the cell defined in the .xyz file

In [ ]:
from collections import OrderedDict

# query AiiDA database for Computers
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': 'cp2k'})
qb.order_by({Code: {'id': 'asc'}})

all_computers = OrderedDict()
all_computers['Please select a computer'] = False
for match in qb.all():
    code = match[1]
    label = match[0]
    all_computers[label] = code

style = {'description_width': '120px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer", options=all_computers,
                            style=style, layout=layout)

int_machines = ipw.IntText(value=1,
                           description='# Nodes',
                           style=style, layout=layout)

text_replica_name = ipw.Text(placeholder='Some cool name',
                             description='Replica Name',
                             style=style, layout=layout)

text_cell = ipw.Text(placeholder='40 40 40',
                     description='Cell',
                     style=style, layout=layout)

text_fixed_atoms = ipw.Text(placeholder='1..10',
                            description='Fixed Atoms',
                            style=style, layout={'width': '60%'})

btn_fixed_atoms = ipw.Button(description='apply (F)',
                            layout={'width': '10%'})

btn_fixed_atoms.on_click(lambda b: refresh_structure_view())

display(
    drop_computer, int_machines,
    text_replica_name, text_cell,
    ipw.HBox([text_fixed_atoms, btn_fixed_atoms])
)

## Step 3: Select type of calculation

**Mixed DFTB**:
Mixed calculation.
Part 1: FIST using an empirical potential (for the slab)
Part 2: DFTB

**Mixed DFT**:
Mixed calculation.
Part 1: FIST using an empirical potential (for the slab)
Part 2: DFT

**Full DFT**:
All of it in DFT.

In [ ]:
toggle_dft = ipw.ToggleButtons(options=['Mixed DFTB', 'Mixed DFT', 'Full DFT'],
                               description='Calculation Type',
                               tooltip='Active: DFT, Inactive: DFTB',
                               style=style, layout=layout)

display(toggle_dft)

## Step 4: Define collective variable
The unit of the spring constant is [eV/unit of colvar^2]. The unit of the targets is defined in the workchain.

In [ ]:
spring_unit = ''

In [ ]:
def colvar_distance_form():
    global spring_unit, target_unit, text_colvar_atoms
    spring_unit = 'eV/angstrom^2'
    target_unit = 'angstrom'
    text_colvar_atoms = ipw.Text(placeholder='1 2 3',
                                 description='Colvar Atoms',
                                 style=style, layout={'width': '60%'})

    btn_colvar_atoms = ipw.Button(description='apply (O)',
                                layout={'width': '10%'})

    btn_colvar_atoms.on_click(lambda b: refresh_structure_view())
    
    return [ipw.HBox([text_colvar_atoms, btn_colvar_atoms]), ]

In [ ]:
def colvar_anglepp_form():
    global spring_unit, target_unit
    global toggle_plane1_type, toggle_plane2_type, text_plane1_def, text_plane2_def
    spring_unit = 'eV/deg^2'
    target_unit = 'deg'
    def on_plane1_type(c):
        text_plane1_def.description = toggle_plane1_type.value
    
    toggle_plane1_type = ipw.ToggleButtons(options=['ATOMS', 'VECTOR'],
                                           description='Plane 1 definition',
                                           style=style, layout=layout)
    toggle_plane1_type.observe(on_plane1_type, 'value')
    
    text_plane1_def = ipw.Text(placeholder='1 2 3',
                               description='Atoms',
                               style=style, layout=layout)
    
    def on_plane2_type(c):
        text_plane2_def.description = toggle_plane2_type.value
    
    toggle_plane2_type = ipw.ToggleButtons(options=['ATOMS', 'VECTOR'],
                                           description='Plane 2 definition',
                                           style=style, layout=layout)
    toggle_plane2_type.observe(on_plane2_type, 'value')
    
    text_plane2_def = ipw.Text(placeholder='1 2 3',
                               description='Atoms',
                               style=style, layout=layout)
    
    return toggle_plane1_type, text_plane1_def, toggle_plane2_type, text_plane2_def

In [ ]:
def render_colvar_form(t):
    # MUST RETURN AN ITERABLE
    if t == 'DISTANCE':
        return colvar_distance_form()
    elif t == 'ANGLE_PLANE_PLANE':
        return colvar_anglepp_form()
    else:
        return (ipw.HTML('NOT IMPLEMENTED'), )

In [ ]:
def on_choose_colvar(c):
    with out_colvar:
        clear_output()
        [display(x) for x in render_colvar_form(drop_colvar_type.value)]

In [ ]:
drop_colvar_type = ipw.Select(
    options={
        'DISTANCE': 'DISTANCE',
        'ANGLE_PLANE_PLANE': 'ANGLE_PLANE_PLANE'
    },
    value='DISTANCE',
    description='Colvar Type',
    style=style, layout=layout
)

drop_colvar_type.observe(on_choose_colvar, 'value')

out_colvar = ipw.Output(layout={'border': '1px solid #ccc'})

text_colvar_targets = ipw.Text(placeholder='0.9 1.3 1.7 2.4',
                               description='Colvar Targets',
                               style=style, layout=layout)

int_spring = ipw.IntText(description='Spring constant',
                         value=75.0,
                         style=style, layout=layout)


display(drop_colvar_type,
        out_colvar,
        text_colvar_targets,
        int_spring)

on_choose_colvar('')

### Generate cp2k inputs

In [ ]:
def generate_cp2k_subsys_colvar():
    cp2k_subsys = { drop_colvar_type.value: {} }
    if drop_colvar_type.value == 'DISTANCE':
        cp2k_subsys[drop_colvar_type.value] = {
            'ATOMS': text_colvar_atoms.value
        }
    elif drop_colvar_type.value == 'ANGLE_PLANE_PLANE':
        repl = {
            'ATOMS': 'ATOMS',
            'VECTOR': 'NORMAL_VECTOR'
        }
        cp2k_subsys[drop_colvar_type.value] = {
            'PLANE': {
                'DEF_TYPE': toggle_plane1_type.value,
                repl[toggle_plane1_type.value]: text_plane1_def.value
            },
            'PLANE  ': {
                'DEF_TYPE': toggle_plane2_type.value,
                repl[toggle_plane2_type.value]: text_plane2_def.value
            }
        }

    return ParameterData(dict=cp2k_subsys)

In [ ]:
def on_submit(b):
    cp2k_code = drop_computer.value
    struct = struct_browser.results.value
    num_machines = Int(int_machines.value)
    replica_name = Str(text_replica_name.value)
    cell = Str(text_cell.value)
    fixed_atoms = Str(text_fixed_atoms.value)

    colvar_targets = Str(text_colvar_targets.value)
    spring = Float(int_spring.value)
    
    calc_type = Str(toggle_dft.value)

    outputs = submit(ReplicaWorkchain,
                     cp2k_code=cp2k_code,
                     structure=struct,
                     num_machines=num_machines,
                     replica_name=replica_name,
                     cell=cell,
                     fixed_atoms=fixed_atoms,
                     colvar_targets=colvar_targets,
                     target_unit=Str(target_unit),
                     spring=spring,
                     spring_unit=Str(spring_unit),
                     subsys_colvar=generate_cp2k_subsys_colvar(),
                     calc_type=calc_type)

    print(outputs)
    the_workfunction = load_node(outputs.pid)
    the_workfunction.description = replica_name

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)

## Evaluate collective variable

In [ ]:
def on_preview_colvar(b):
    with output_preview:
        t = deepcopy(struct_browser.results.value.get_ase())
        clear_output()

        a_list = [int(x) for x in text_colvar_def_preview.value.split()]
        for a in a_list:
            t[a-1].symbol = 'O'
        
        print 'List of atoms {}'.format(a_list)
        if drop_colvar_type_preview.value == 'DISTANCE':
            d = t[a_list[0]-1].position-t[a_list[1]-1].position
            print "Distance vector ({}, {}): {}".format(a_list[0], a_list[1], d)
            print "Distance: {}".format(np.linalg.norm(d))

        if drop_colvar_type_preview.value == 'ANGLE_PLANE_PLANE':
            v1 = t[a_list[0]-1].position - t[a_list[1]-1].position
            v2 = t[a_list[0]-1].position - t[a_list[2]-1].position
            
            cosine = np.dot(np.cross(-v1, v2), [0,0,1])/np.linalg.norm(np.cross(-v1, v2))
            angle = np.arccos(cosine)*180./np.pi
            print 'Cosine: {}'.format(cosine)
            print 'Angle: {}'.format(angle)

        if toggle_ngl.value:
            display(nglview.show_ase(t))
        else:
            display(display_thumbnail(render_thumbnail(t)))
        
drop_colvar_type_preview = ipw.Select(
    options={
        'DISTANCE': 'DISTANCE',
        'ANGLE_PLANE_PLANE': 'ANGLE_PLANE_PLANE'
    },
    value='ANGLE_PLANE_PLANE',
    description='Colvar Type',
    style=style, layout=layout
)

drop_colvar_type.observe(on_preview_colvar, 'value')

text_colvar_def_preview = ipw.Text(placeholder='1 2 3',
                                   description='Colvar Atoms',
                                   style=style, layout=layout)

toggle_ngl = ipw.ToggleButton(description='nglview',
                              style=style, layout=layout)

btn_preview = ipw.Button(description='Try')
output_preview = ipw.Output()
btn_preview.on_click(on_preview_colvar)
display(drop_colvar_type_preview, text_colvar_def_preview, toggle_ngl, btn_preview, output_preview)